# Optional Lab: Multiple Variable Linear Regression
# 2 Problem Statement

In [7]:
X_train = [2104 5 1 45; 1416 3 2 40; 852 2 1 35]
y_train = [460, 232, 178];

## 2.1 Matrix X containing our examples

In [10]:
# data is stored in numpy array/matrix
println("X Shape: $(size(X_train)), X Type: $(eltype(X_train))")
println(X_train)
println("y Shape: $(size(y_train)), y Type:$(eltype(y_train))")
println(y_train)

X Shape: (3, 4), X Type: Int64
[2104 5 1 45; 1416 3 2 40; 852 2 1 35]
y Shape: (3,), y Type:Int64


[460, 232, 178]


## 2.2 Parameter vector w, b

In [11]:
b_init = 785.1811367994083
w_init = [ 0.39133535, 18.75376741, -53.36032453, -26.42131618]
println("w_init shape: $(size(w_init)), b_init type: $(eltype(b_init))")

w_init shape: (4,), b_init type: Float64


# 3 Model Prediction With Multiple Variables
## 3.1 Single Prediction element by element

In [15]:
range(0, 9)

0:9

In [16]:
function predict_single_loop(x, w, b)
    """
    single predict using linear regression

    Args:
    x (ndarray): Shape (n,) example with multiple features
    w (ndarray): Shape (n,) model parameters    
    b (scalar):  model parameter     

    Returns:
    p (scalar):  prediction
    """
    n = size(x)[1]
    p = 0
    for i in 1:n
        pⁱ = x[i] * w[i]  
        p = p + pⁱ
    end
    p + b
end

predict_single_loop (generic function with 1 method)

In [20]:
# get a row from our training data
x_vec = X_train[1, :]
println("x_vec shape $(size(x_vec)), x_vec value: $(x_vec)")

# make a prediction
f_wb = predict_single_loop(x_vec, w_init, b_init)
println("f_wb shape $(size(f_wb)), prediction: $f_wb")

x_vec shape (4,), x_vec value: [2104, 5, 1, 45]
f_wb shape (), prediction: 459.9999976194083
